In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail()

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df=df.sample(5000).drop("link",axis=1).reset_index(drop=True)

In [8]:
df['text'][0]

"There's only us, there's only this  \r\nForget regret, or life is your's to miss  \r\nNo other path, no other way  \r\nNo day but today  \r\n  \r\nThere's only us, only tonight  \r\nWe must let go to know what's right  \r\nNo other road, No other way  \r\nNo day but today  \r\n  \r\nI can't control my destiny  \r\nI trust my soul, my only goal  \r\nIs just to be  \r\n  \r\nThere's only now, there's only here  \r\nGive in to love or live in fear  \r\nNo other path, No other way  \r\nNo day but today  \r\n  \r\nThere's only us, There's only this  \r\nForget regret, or life is your's to miss  \r\nNo other road, no other way  \r\nNo day but today  \r\n  \r\nNo day but today  \r\n(No day but today)  \r\nNo day but today  \r\n(No day but today)  \r\nNo day but today  \r\n(No day but today)  \r\nNo day but today\r\n\r\n"

In [9]:
df.shape

(5000, 3)

Text Cleaning / text Preprocessing

In [10]:
df['text']=df['text'].str.lower().replace(r'^\w\s', '').replace(r'\n', '' ,regex=True)

In [11]:
df.tail(5)

,artist,song,text
4995,Nickelback,Shakin' Hands,"(hey, hey) she had her eyes on the prize as th..."
4996,Donna Summer,Born To Die,just a baby \ron a doorstep \rit's how they ...
4997,Foo Fighters,Throwing Needles,what's there at the door that changed my life ...
4998,Dewa 19,Sweetest Place,this is the sweetest place \rbaring my feelin...
4999,Vince Gill,Nothing Like A Woman,oh there's nothin' like a woman \rwhen you're...


In [12]:
import nltk
from nltk.stem.porter import PorterStemmer

In [13]:
stemmer=PorterStemmer()

In [14]:
def token(txt):
    token=nltk.word_tokenize(txt)
    a=[stemmer.stem(w) for w in token]
    return " ".join(a)

In [15]:
token("you are beautiful,beauty")

'you are beauti , beauti'

In [16]:
df["text"].apply(lambda x: token(x))

0       there 's onli us , there 's onli thi forget re...
1       did you get your race card ? i did n't get my ...
2       i feel so close to you 'caus you know you alwa...
3       i get deep , i get deep , i get deep , i get d...
4       i found a black beret on the street today it w...
                              ...                        
4995    ( hey , hey ) she had her eye on the prize as ...
4996    just a babi on a doorstep it 's how they found...
4997    what 's there at the door that chang my life w...
4998    thi is the sweetest place bare my feel i 'm we...
4999    oh there 's nothin ' like a woman when you 're...
Name: text, Length: 5000, dtype: object

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
tfid=TfidfVectorizer(analyzer='word',stop_words='english')

In [19]:
matrix=tfid.fit_transform(df['text'])

In [20]:
similer=cosine_similarity(matrix)

In [21]:
similer[0]

array([1.        , 0.00594108, 0.10714463, ..., 0.06753898, 0.01494817,
       0.        ])

In [23]:
df[df['song']=='Born To Die'].index[0]

4996

Recommender function

In [24]:
def recommender(song_name):
    idx=df[df['song']==song_name].index[0]
    distance=sorted(list(enumerate(similer[idx])),reverse=True,key=lambda x:x[1])
    song=[]
    for s_id in distance[1:21]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [27]:
recommender('Born To Die')

['Just Born',
 'Born To Lose',
 'Born To Lose',
 "We're All Gonna Die",
 'Born To Be Somebody',
 'Children Go Where I Send Thee',
 'The Beloved',
 'Someday Man',
 'Born To Be Wild',
 "I Can't Give You More",
 'I Am A Man Of Constant Sorrow',
 'House On The Ocean',
 'Our Diabolical Rapture',
 "Please Don't Die",
 'Everybody Wants To Go To Heaven',
 "I'm A Woman",
 'I Am A Man Of Constant Sorrow',
 'Born As Ghosts',
 'Only The Good Die Young',
 'Born To Hand Jive']

In [28]:
import pickle

In [29]:
pickle.dump(similer, open('similarity.pkl', 'wb'))

In [30]:
pickle.dump(df, open("df.pkl", "wb"))